# Achievements effect on Retention to third month in ecosystem

We call 'Uncofoundedness' a scenario where a treatment is not randomly assigned to participants, so confounders effect on treatment assignment and outcome.

Treatment - purchase in one category.

We will test hypothesis:

$H_o$ - There is no difference in retention to third month between treatment and control groups.

$H_a$ - There is a difference in LTV between treatment and control groups.

In [1]:
from causalis.scenarios.unconfoundedness.dgp import generate_obs_hte_binary_26

data = generate_obs_hte_binary_26(return_causal_data=False, include_oracle=True)
data.head()


,y,d,tenure_months,avg_sessions_week,spend_last_month,age_years,prior_purchases_12m,support_tickets_90d,premium_user,mobile_user,weekend_user,email_opt_in,referred_user,m,m_obs,tau_link,g0,g1,cate
0,0.0,0.0,28.814654,1.0,78.459423,50.392490,4.0,2.0,0.0,1.0,1.0,1.0,0.0,0.136804,0.136804,-0.075690,0.259586,0.245305,-0.014281
1,1.0,1.0,10.987367,3.0,38.652698,31.652666,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.157599,0.157599,0.781429,0.592325,0.760425,0.168101
2,0.0,1.0,40.678212,9.0,98.950760,48.634055,4.0,5.0,0.0,1.0,0.0,0.0,0.0,0.165401,0.165401,0.209518,0.043862,0.053538,0.009676
3,0.0,1.0,14.331764,5.0,27.386588,42.502641,3.0,3.0,1.0,1.0,1.0,0.0,0.0,0.158897,0.158897,0.630457,0.148391,0.246602,0.098211
4,0.0,1.0,21.480304,2.0,119.753960,35.311382,3.0,0.0,0.0,1.0,1.0,1.0,0.0,0.169943,0.169943,0.346384,0.527043,0.611748,0.084704


In [2]:
print(f"Ground truth ATE is {data['cate'].mean()}")
print(f"Ground truth ATTE is {data[data['d'] == 1]['cate'].mean()}")

Ground truth ATE is 0.08155183943650529
Ground truth ATTE is 0.10123794590017934


In [3]:
from causalis.data_contracts import CausalData

causaldata = CausalData(df = data,
                        treatment='d',
                        outcome='y',
                        confounders=['tenure_months', 'avg_sessions_week', 'spend_last_month', 'age_years', 'income_monthly', 'prior_purchases_12m', 'support_tickets_90d', 'premium_user', 'mobile_user', 'urban_resident', 'referred_user'])
causaldata

ValidationError: 1 validation error for CausalData
  Value error, Column 'income_monthly' specified as confounders does not exist in the DataFrame. [type=value_error, input_value={'df':          y    d  t...dent', 'referred_user']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [ ]:
from causalis.shared import outcome_stats
outcome_stats(causaldata)

In [ ]:
from causalis.shared import outcome_plots
outcome_plots(causaldata)

In [ ]:
from causalis.shared import confounders_balance

confounders_balance(causaldata)

# Inference

In [ ]:
from causalis.scenarios.unconfoundedness import IRM

model = IRM().fit(causaldata)

### Math Explanation of the IRM Model and ATTE Estimand

The **Interactive Regression Model (IRM)** is a flexible framework used in Double Machine Learning (DML) to estimate treatment effects. Unlike linear models, it allows the treatment effect to vary with confounders $X$ (interaction) and makes no parametric assumptions about the functional forms of the outcomes.

We write $W=(Y,D,X)$ for an observation, where $D\in\{0,1\}$ is treatment and $Y$ is the observed outcome.



#### 1. Nuisance Functions
The IRM framework relies on three "nuisance" components estimated from the data:
*   **Outcome Regression (Control):** $g_0(X) = \mathbb{E}[Y | X, D=0]$
*   **Outcome Regression (Treated):** $g_1(X) = \mathbb{E}[Y | X, D=1]$
*   **Propensity Score:** $m(X) = \mathbb{P}(D=1 | X)$

Let $p = \mathbb{P}(D=1) = \mathbb{E}[D]$ denote the overall treatment rate (estimated by the sample mean of $D$).

In the provided implementation (`irm.py`), these are estimated using cross-fitting (splitting data into folds) to avoid overfitting bias.



#### 2. ATTE (Average Treatment Effect on the Treated)
The **Average Treatment Effect on the Treated (ATTE)** measures the impact of the treatment specifically on those individuals who received it:
$$\theta_{ATTE} = \mathbb{E}[Y(1) - Y(0) \mid D=1]$$

Under **unconfoundedness**, $(Y(1),Y(0)) \perp D \mid X$, and overlap $0 < m(X) < 1$, this is identified from observed data.



#### 3. The Orthogonal Score
DML uses a **Neyman-orthogonal score** $\psi$ to ensure the estimator is robust to small errors in the nuisance function estimates. The score for ATTE is defined as:
$$\psi(W; \theta, \eta) = \psi_b(W; \eta) + \psi_a(W; \eta)\theta$$

To match the implementation in `irm.py`, define:
*   **Residuals:** $u_0 = Y - g_0(X)$, $u_1 = Y - g_1(X)$
*   **IPW terms:** $h_1 = \frac{D}{m(X)}$, $h_0 = \frac{1-D}{1-m(X)}$
*   **Weights (ATTE):** $w = \frac{D}{p}$ and $\bar{w} = \frac{m(X)}{p}$ (the normalized form with $\mathbb{E}[w]=1$)

Then:
\begin{aligned}
\psi_a(W;\eta) &= -w = -\frac{D}{p} \\
\psi_b(W;\eta) &= w\,(g_1(X)-g_0(X)) + \bar{w}\,(u_1 h_1 - u_0 h_0)
\end{aligned}

(If `normalize_ipw=True`, the code rescales $h_1$ and $h_0$ to have mean 1.)



#### 4. Final Estimation (Step-by-step simplification)
For brevity, write $m = m(X)$, $g_0 = g_0(X)$, and $g_1 = g_1(X)$. Plug in $w, \bar{w}, h_1, h_0$:


\begin{aligned}
\psi_b
&= \frac{D}{p}(g_1-g_0)
  + \frac{m}{p}\left[\frac{D}{m}(Y-g_1) - \frac{1-D}{1-m}(Y-g_0)\right] \\
&= \frac{D}{p}(g_1-g_0) + \frac{D}{p}(Y-g_1) - \frac{m}{p}\frac{1-D}{1-m}(Y-g_0) \\
&= \frac{D}{p}(Y-g_0) - \frac{m}{p}\frac{1-D}{1-m}(Y-g_0).
\end{aligned}


So the $g_1(X)$ terms cancel, and the ATTE score depends only on $g_0(X)$ and $m(X)$.

The estimator solves $\mathbb{E}[\psi(W;\theta,\eta)]=0$:
\begin{aligned}
\hat{\theta}_{ATTE}
&= \frac{\mathbb{E}[\psi_b]}{\mathbb{E}[-\psi_a]}
= \frac{\mathbb{E}[\psi_b]}{\mathbb{E}[D/p]}
= \mathbb{E}[\psi_b].
\end{aligned}

Equivalently,
$$\hat{\theta}_{ATTE} = \mathbb{E}\left[\frac{D}{p}(Y-g_0(X)) - \frac{m(X)}{p}\frac{1-D}{1-m(X)}(Y-g_0(X))\right].$$



In [ ]:
result = model.estimate(score='ATTE')
result.summary()

In [ ]:
result

In [ ]:
from causalis.scenarios.unconfoundedness.refutation import *
rep = run_overlap_diagnostics(res=result)
rep["summary"]

In [ ]:
plot_m_overlap(result.diagnostic_data)

In [ ]:
from causalis.scenarios.unconfoundedness.refutation.score.score_validation import run_score_diagnostics
rep_score = run_score_diagnostics(res=result)
rep_score["summary"]

In [ ]:
print_sutva_questions()

In [ ]:
from causalis.scenarios.unconfoundedness.refutation.uncofoundedness.uncofoundedness_validation import run_uncofoundedness_diagnostics

rep_uc = run_uncofoundedness_diagnostics(res=result)
rep_uc['summary']

In [ ]:
from causalis.scenarios.unconfoundedness.refutation.uncofoundedness.sensitivity import (
    sensitivity_analysis, sensitivity_benchmark
)

sensitivity_analysis(result, r2_y=0.01, r2_d=0.01, rho=1.0, alpha=0.05)

In [ ]:
sensitivity_benchmark(result.diagnostic_data, benchmarking_set =['tenure_months'])